In [56]:
import pandas as pd
import numpy as np

# Chapter 6

## Data Loading, Storage, and File Formats

## 6.1 Reading and Writing Data in Text Format

pandas has a number of functions for reading tabular data, but will most likely use read_csv and read_table

optional arguments for these functions include: indexing, type inference and data conversion, datetime parsing, iterating, and unclean data issues

handling dates and other custom types can require extra effort. start with a small comma-separated (CSV) file:

In [1]:
!cat examples/ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [4]:
df = pd.read_csv('examples/ex1.csv')

In [5]:
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [6]:
pd.read_table('examples/ex1.csv', sep=',') #used read-table and specified the deliminator

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [7]:
!cat examples/ex2.csv #file will not always have a header row

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [8]:
pd.read_csv('examples/ex2.csv', header=None) #allow pandas to assign default column names

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [9]:
pd.read_csv('examples/ex2.csv', names=['a', 'b', 'c', 'd', 'message']) #assign column names

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [10]:
names = ['a', 'b', 'c', 'd', 'message']

In [11]:
pd.read_csv('examples/ex2.csv', names=names, index_col='message') #setting message column to be the index

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


if you want to form a hierarchical index from multiple columns, pass a list of column numbers or names:

In [12]:
!cat examples/csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [13]:
parsed = pd.read_csv('examples/csv_mindex.csv', index_col=['key1', 'key2'])

In [14]:
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

in some cases a table might not have a fixed delimiter, using whitespace or some other pattern to seaparate fields. for example:


In [15]:
list(open('examples/ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

the fields here are separated by a variable amount of whitespace. in these cases, you can pass a regular expression as a delimiter for read-table. use the reg:ex \s+

In [17]:
result = pd.read_table('examples/ex3.txt', sep='\s+')

In [18]:
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


In [20]:
!cat examples/ex4.csv


# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [22]:
pd.read_csv('examples/ex4.csv', skiprows=[0, 2, 3]) #skipping rows

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


handling missing values- by default, pandas uses a set of commonly occuring sentinels such as NA and NULL

In [23]:
!cat examples/ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo

In [24]:
result = pd.read_csv('examples/ex5.csv')

In [25]:
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [26]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


the na_values option can take either a list or set of strings to consider missing values:

In [27]:
result = pd.read_csv('examples/ex5.csv', na_values=['Null'])

In [28]:
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [32]:
sentinels = {'message': ['foo', 'NA'], 'something': ['two']} 

#differente NA sentinels can be specified for each column in a dict:

In [33]:
pd.read_csv('examples/ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


### Reading Text Files in Pieces

when processing large files, may want to only read a small piece or iterate through a small piece first. 

In [34]:
pd.options.display.max_rows = 10 #make pandas display settings compact

In [35]:
result = pd.read_csv('examples/ex6.csv')

In [36]:
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


In [37]:
pd.read_csv('examples/ex6.csv', nrows=5) #specify number of rows you want to read with nrows

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


In [38]:
chunker = pd.read_csv('examples/ex6.csv', chunksize=1000) #read a file in pieces use chunksize as a number of rows

In [39]:
chunker

the TextParser object returned by read_csv allows you to iterate over the parts of the file according to the chunksize. for example, we can iterate over ex6.csv, aggregating the value counts in the 'key' column:

In [41]:
chunker = pd.read_csv('examples/ex6.csv', chunksize =1000)

tot = pd.Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)
    
tot = tot.sort_values(ascending=False)

/var/folders/rw/1rnjc7b90vd_mr3753j04_s40000gp/T/ipykernel_30284/3840960893.py:3: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  tot = pd.Series([])


In [42]:
tot[:10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

TextParser is also equipped with a get_chunk method

### Writing Data to Text Format

Data can be exported to a delimited format

In [43]:
data = pd.read_csv('examples/ex5.csv')

In [44]:
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [45]:
data.to_csv('examples/out.csv') #using DataFrame's to_csv method to write the data out to a CSV file

In [46]:
!cat examples/out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


_writing to sys.stdout prints the text result to the console):_

In [47]:
import sys

In [48]:
data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


missing values appear as empty strings in the output. denote them by some other sentinel value:

In [49]:
data.to_csv(sys.stdout, na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


In [51]:
data.to_csv(sys.stdout, index=False, header=False) #disabling row and column labels

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


In [53]:
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c']) #write only a subset of the columns

a,b,c
1,2,3.0
5,6,
9,10,11.0


In [54]:
dates = pd.date_range('1/1/2000', periods=7) #Series CSV Method here and below

In [57]:
ts = pd.Series(np.arange(7), index=dates)

In [58]:
ts.to_csv('examples/tseris.csv')

In [59]:
!cat examples/tseries.csv

2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
2000-01-06,5
2000-01-07,6


### Working with Delimited Formats